# Scaling Data

Sometimes prior to fitting a model or running an algorithm we will need to scale our data. This is particularly true when some features (columns of $X$) are on vastly different scales than others. In this notebook we will demonstrate how to scale.

## What we will accomplish

In this notebook we will:
- Introduce the concept of scaling your data,
- Demonstrate the `StandardScaler` object in `sklearn`,
- Discuss `fit`, `transform` and `fit_transform` and
- Show the scaling process using `sklearn`.

Before we get started let's generate some data.

In [4]:
import numpy as np

In [5]:
## Make some data
## Notice that the columns of X have vastly different scales
rng = np.random.default_rng(440)
X = np.zeros((1000, 4))
X[:, 0] = 1000 * rng.standard_normal(1000)
X[:, 1] = rng.random(1000) - 10
X[:, 2] = rng.integers(-250, 150, 1000)
X[:, 3] = 10 * rng.standard_normal(1000) - 75

In [6]:
## demonstrating the different scales
print("mean of X1:",np.mean(X[:,0]))
print("variance of X1:",np.var(X[:,0]))
print()

print("mean of X2:",np.mean(X[:,1]))
print("variance of X2:",np.var(X[:,1]))
print()

print("mean of X3:",np.mean(X[:,2]))
print("variance of X3:",np.var(X[:,2]))
print()

print("mean of X4:",np.mean(X[:,3]))
print("variance of X4:",np.var(X[:,3]))

mean of X1: -14.025444352510618
variance of X1: 979949.8838817807

mean of X2: -9.48565129185406
variance of X2: 0.08777857572505747

mean of X3: -57.413
variance of X3: 13659.232431

mean of X4: -75.03513575644826
variance of X4: 99.25896607423117


We can see that the columns of `X` have very different scales. We will soon learn some algorithms whose results can be greatly distorted when this happens.

The main approach to fixing this issue is to scale the data so they are all on the same scale.

## Standardizing your data

While there are a few different ways to scale data, one of the most common is to <i>standardize</i> it. When you standardize a variable, $x$, you apply the following transformation:

$$
x_\text{scaled} = \frac{x - \text{mean}(x)}{\text{standard deviation}(x)},
$$

if you have taken a statistics course (or used $Z$-tables), this should look familiar. This is precisely the transformation applied to turn any arbitary normal random variable into a <i>standard normal</i> random variable, hence the term <i>standardizing</i>.

Standardizing your data will transform it to have mean $0$ and standard deviation $1$.

### `StandardScaler`

We could do this by hand using `numpy`, but that will quickly become tedious. `sklearn` provides a nice `scaler` object called `StandardScaler` that will perform this on all columns of your data set, and has functionality that plays nicely with train test splits. Here is the documentation <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html">https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html</a>.

In [7]:
## import StandardScaler
from sklearn.preprocessing import StandardScaler

In [8]:
## Make a scaler object
scaler = StandardScaler()

## fit the scaler
scaler.fit(X)

StandardScaler()

In [9]:
## scale the data, i.e. transform it
X_scale = scaler.transform(X)

In [10]:
## Checking the scaled means and variances
print("mean of standardized X1:",np.mean(X_scale[:,0]))
print("variance of standardized X1:",np.var(X_scale[:,0]))
print()

print("mean of standardized X2:",np.mean(X_scale[:,1]))
print("variance of standardized X2:",np.var(X_scale[:,1]))
print()

print("mean of standardized X3:",np.mean(X_scale[:,2]))
print("variance of standardized X3:",np.var(X_scale[:,2]))
print()

print("mean of standardized X4:",np.mean(X_scale[:,3]))
print("variance of standardized X4:",np.var(X_scale[:,3]))

mean of standardized X1: 0.0
variance of standardized X1: 1.0000000000000007

mean of standardized X2: 9.29833987584061e-15
variance of standardized X2: 1.0000000000000004

mean of standardized X3: 1.2434497875801754e-17
variance of standardized X3: 1.000000000000002

mean of standardized X4: -3.661071446003916e-15
variance of standardized X4: 0.9999999999999991


#### `fit`, `transform` and `fit_transform` & scaling for train test splits

You may be wondering what `fit`, `transform` and `fit_transform` do. Let's describe:
- `fit` performs a fit of the `scaler` object, for `StandardScaler` this means finding the mean and standard deviation of each column and storing it. `fit` must be called <i>before</i> `transform`.
- `transform` is what actually performs the scaling, for `StandardScaler` this means substracting the respective means and dividing by the respective standard deviations for each column. `transform` must be called <i>after</i> `fit`.
- `fit_transform` does this all in one fell swoop, i.e. it fits the `scaler` object then uses the fit to transform the data.

##### Why do we need anything other than `fit_transform`?

Excellent question! In the example above we probably could have just used `fit_transform`.

However, this is because we were not dealing with train test splits, validation sets or cross-validation. We consider scaling the data part of the model, meaning the algorithm/model was fit using the data scaled with the training set's means and standard deviations. For example, if we scaled data prior to fitting a linear regression model, then $\hat{\beta}$ was found using the data scaled according to the training set (using the means and standard deviations of the training set columns). To assess how that particular model performs we must also scale any validation set, cross-validation holdout set or test set using the same exact scaling (i.e. using the means and standard deviations from the training set).

Let's illustrate what I mean with a quick final example.

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test = train_test_split(X,
                                      shuffle=True,
                                      random_state=614,
                                      test_size=.2)

In [13]:
## Fit then transform the training set
scaler_new = StandardScaler()

scaler_new.fit(X_train)

X_train_scale = scaler_new.transform(X_train)

## alternatively I could do
## X_train_scale = scaler_new.fit_transform(X_train), Why?
## because this is the training set, so it is okay to run fit_transform.

<i>Imagine we build a model here.</i>

In [14]:
## Transform the test set
## DO NOT refit the scaler!
X_test_scale = scaler_new.transform(X_test)


In [15]:
X_test_scale

array([[ 0.67111013,  1.43693248,  1.35021459, -0.89976069],
       [ 0.91040691, -0.45561197, -0.85643417, -0.24702929],
       [ 0.40234974,  1.29161749,  1.56403715, -1.4422597 ],
       [-0.9177747 ,  1.42299846, -0.64261161, -0.8552297 ],
       [-1.15156642, -0.60380916, -1.33539669,  0.46151387],
       [-1.53904172,  0.63679008,  0.12714958, -1.31754538],
       [-0.01016625,  0.25885077,  1.19626235, -0.6033813 ],
       [-0.97798214,  1.36894554, -0.56563549, -1.01100684],
       [-1.947198  ,  1.52566765, -1.13867994, -1.90434754],
       [ 0.82505491,  0.7625549 ,  1.10218043,  0.3561862 ],
       [ 0.49791925, -1.42057377, -1.18999735,  0.08472803],
       [-2.14387332, -1.40332655,  1.13639204,  0.34763869],
       [-0.39808911, -1.71845048,  1.53837844,  0.13116898],
       [ 0.11796469, -1.24420924,  0.9995456 ,  2.52219762],
       [ 0.60280292, -0.86973717, -0.05246136,  0.19836516],
       [ 0.70371146,  0.48139199, -0.91630448, -1.15606754],
       [-0.7894837 , -0.

<i>Imagine we test model performance on the test set here.</i>

## Other scaler objects

`sklearn` has more scalers than just `StandardScaler`, which you can find at this link, <a href="https://scikit-learn.org/stable/modules/preprocessing.html">https://scikit-learn.org/stable/modules/preprocessing.html</a>. Moreover, some `sklearn` models have their own input arguments that will handle scaling for you when set `= True`.

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)